This job is to scrape all of the reviews from this URL - https://www.toothssenger.com/118276-ofallon-dentist-dr-edward-logan#read_review.

They just need to be in a CSV file.

We will need the following information.

Star Rating - Primary, Facilities, Service, Painless, Results, Cost
Name
Date
Review
About Front Desk
About Hygeienist
My Experience
Duration
I know this doctor  

In [56]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

url = 'https://www.toothssenger.com/118276-ofallon-dentist-dr-edward-logan#read_review'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc, "lxml")

In [69]:
blocks = soup.find_all("div", attrs={"class" : "jsReviewItem"}, limit=20)
one_block = soup.find("div", attrs={"class" : "jsReviewItem"})

records = []

#TO DO: handle infinite scrolling on webpage

for block in blocks:
    
    #review number
    review_num = block.select("div b")[0]
    if len(review_num) > 0:
        review_num = review_num.text.strip()
    else:
        review_num = "NaN"
    
    #reviewer name
    name = block.select("span")[1]
    if len(name) > 0: 
        name = name.text.strip()
    else:
        name = "NaN"
    
    #date published
    date = block.select('span[itemprop="datePublished"]')
    if len(date) > 0:
        date = date[0].text.strip()
    else:
        name = "NaN"
    
    #review text 
    review = block.find("span", {"itemprop":"reviewBody"})
    if review is not None:
        review = review.text.strip()    
    else: 
        review = "NaN"
     
    #select ratings tag and count the number of icons for each rating type
    ratings = block.select('div[class="mb-0-20"]')
    
    if len(ratings) > 0:
    
        #facilities ratings
        fac = ratings[0]
        fac_rating = (len(list(fac.find_all("i"))))
    
        #service rating
        serv = ratings[1]
        serv_rating = (len(list(serv.find_all("i"))))
    
        #painless rating
        painless = ratings[2]
        painless_rating = (len(list(painless.find_all("i"))))

        #results rating
        results = ratings[3]
        results_rating = (len(list(results.find_all("i"))))

        #cost rating
        cost = ratings[4]
        cost_rating = (len(list(cost.find_all("i"))))
        
    else:
        ratings = "NaN"
    
    records.append((review_num, name, date, review, fac_rating, serv_rating, painless_rating, cost_rating))   
     
print(records[0])

#about front desk 

#about hygeienist 

#my experience duration 

#i know this doctor


('Review 1', 'Smiling Patient', 'Sept. 26, 2016, 9:34 p.m.', "I had a long time crown that lifted off. I was so worried because I was leaving the country on vacation the next day.  My message got to Dr. Logan and he met me at his office on a Saturday to reset my crown!!   I just called to get an appointment for when I returned from vacation and his office staff and Dr. Logan  arranged this and called me back to help me out!  I was so thrilled and thankful!  This is a true testament to Dr. Logan's attention and care to his patients!", 4, 4, 4, 2)


In [70]:
df = pd.DataFrame(records, columns=["Review #","Name","Date","Review","Facilities Rating","Service Rating","Painless Rating",
                                    "Cost Rating"])


In [71]:
df.head()

,Review #,Name,Date,Review,Facilities Rating,Service Rating,Painless Rating,Cost Rating
0,Review 1,Smiling Patient,"Sept. 26, 2016, 9:34 p.m.",I had a long time crown that lifted off. I was...,4,4,4,2
1,Review 2,Smiling Patient,"May 15, 2014, 1:38 p.m.",Dr. Amorin at Dr. Logan's office is wonderful!...,4,4,4,2
2,Review 3,Sho,"May 14, 2014, 10:19 p.m.",My review is about Dr. Logan's office and the ...,4,4,4,2
3,Smiling Patient,NaN,[],NaN,4,4,4,2
4,Review 4,Smiling Patient,"Feb. 11, 2014, 11:38 a.m.",I recieved dental implants for 6 of my teeth b...,4,4,4,3


In [72]:
df.tail()

,Review #,Name,Date,Review,Facilities Rating,Service Rating,Painless Rating,Cost Rating
5,Review 5,SLC,"Sept. 1, 2012, 8:20 p.m.",The office staff was friendly and knowledgeabl...,3,4,4,2
6,Review 6,Smiling Patient,"Jan. 10, 2011, 11:21 a.m.","Dr Logan has been my dentist for 6 years, ever...",4,4,4,3
7,Review 7,Smiling Patient,"Oct. 15, 2009, 12:41 p.m.",I first met Dr. Logan when I was looking for a...,4,4,4,3
8,Review 8,marc,"Oct. 5, 2009, 11:58 a.m.",I no longer fear going to the dentist!\nDr. L...,4,4,4,2
9,Review 9,Smiling Patient,"Sept. 8, 2009, 8:39 p.m.","Great dentist, great office. Our family was a...",4,4,4,3


In [74]:
df.to_csv("dentist_reviews.csv")